In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error


path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-20 22:02:46,499 utils 400 [INFO]    [logger_func] start 


In [2]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(model_path):
    base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    tmp = utils.read_pkl_gzip(model_path)
    if 'pred_mean' in tmp.columns:
        pred_col = 'pred_mean'
    elif 'prediction' in tmp.columns:
        pred_col = 'prediction'
    tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
    
    train['pred'] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train['pred'].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================


#========================================================================
# 相関

def get_corr_ensemble(path, base, out_part = ''):
    '''
    1. ベースモデルとの相関を個別に取得する
    2. ベースモデルにpath_listのモデルの予測をjoinし、相関行列を取得する
    '''
    
    if key in base.columns:
        base = base.set_index(key)
    
    #========================================================================
    # 個別に相関を見ていく場合
    if str(type(path)).count('str'):
    
        if path.count('CV1'):
            return 0, ''
        if path.count('binary'):
            return 0, ''
        tmp = utils.read_pkl_gzip(path)
    
        try:
            tmp = tmp.reset_index()[[key, 'pred_mean']].set_index(key)
            base['tmp_pred'] = tmp.reset_index()['pred_mean']
        except KeyError:
            tmp = tmp.reset_index()[[key, 'prediction']].set_index(key)
            base['tmp_pred'] = tmp['prediction']
    
        if path.count('LB'):
            try:
                cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
            except AttributeError:
                return 0, ''
        else:
            cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
    
        corr = np.corrcoef(base['base_pred'], base['tmp_pred'].values).min()
    
        if corr>0 and corr<0.98:
            logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
            
            return corr, path
        else:
            return 0, ''
    #========================================================================
    
    #========================================================================
    # まとめて相関行列を見たい場合
    elif str(type(path)).count('list'):
        path_list = path
        for path in path_list:
        
            if out_part=='all':
                if path.count('CV-1') or path.count('binary'):
                    continue
            elif out_part=='rm_out':
                if path.count('CV-3') or path.count('binary'):
                    continue
            elif out_part=='binary':
                if path.count('CV-3') or path.count('CV-1'):
                    continue
            tmp = utils.read_pkl_gzip(path)

            if 'pred_mean' in tmp.columns:
                pred_col = 'pred_mean'
            elif 'prediction' in tmp.columns:
                pred_col = 'prediction'
                
            tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
                
            if path.count('LB'):
                try:
                    cv2 = re.search(r'CV([^/.]*)_LB', path.replace('.', '-')).group(1)
                except AttributeError:
                    print(path.replace('.', '-'))
                    continue
            else:
                cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
                
            base[f'pr_{cv2[:9]}'] = tmp[pred_col]
            
        if out_part=='rm_out':
            base = base[base[target]>-30]
        drop_cols = [col for col in base.columns if col.count('term') or col.count('no_out_flg') or col in ignore_list or col.count('clf')]
        base.drop(drop_cols, axis=1, inplace=True)
        base.sort_index(axis=1, inplace=True)
        mx_corr = base.corr(method='pearson')
#         mx_cor = mx_corr.drop(drop_cols, axis=1).loc[~mx_corr.index.isin(drop_cols)]
        
        return mx_corr
#========================================================================

In [18]:
is_clf = 0
base = utils.read_df_pkl('../input/base_no_out*')[[key, target, 'clf_pred', 'no_out_flg']].set_index(key)
# LB3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz').set_index(key)
# base['lb3664'] = LB3664['prediction']
#========================================================================
# rm_out以外の部分の相関を見る
# base = base[base['clf_pred']>=0.01]
#========================================================================
# ens_list = glob.glob('../stack/*lgb*all*CV3*.gz')
# ens_list = glob.glob('../model/LB3664_set/*_lgb_*.gz')
ens_list = glob.glob('../ensemble/lgb_ensemble/*_lgb_*.gz')
ens_list = glob.glob('../ensemble/rm_outlier_ensemble/*_lgb_*.gz')
out_part = 'all'
# ens_list += glob.glob('../stack/0213_2*no_out*.gz') + glob.glob('../stack/0214_*no_out*.gz')

if is_clf:
    out_part = 'binary'
#     ens_list = glob.glob('../stack/0214*NN*-clf*.gz')
    ens_list = glob.glob('../ensemble/binary_ensemble/*.gz')
    base[target] = base[target].map(lambda x: 1 if x<-30 else 0)
top_corr = get_corr_ensemble(path=ens_list, base=base, out_part=out_part)
print(top_corr.shape)
display(top_corr.head())
# top_corr.to_csv('../output/0214_07_elo_rm_out_corr_ensemble_heatmap.csv', index=True)

100%|██████████| 1/1 [00:00<00:00,  4.36it/s]


(18, 18)


,pr_1-5486783,pr_1-5494467,pr_1-5497746,pr_1-5500177,pr_1-5500195,pr_1-5500491,pr_1-5500979,pr_1-5520199,pr_1-5520200,pr_1-5520218,pr_1-5520923,pr_1-5530348,pr_1-5530796,pr_1-5530868,pr_1-5530958,pr_1-5540396,pr_1-5540891,pr_1-5552829
pr_1-5486783,1.000000,0.987609,0.989740,0.986183,0.986011,0.988688,0.987826,0.965187,0.967044,0.965100,0.981394,0.963643,0.963676,0.971796,0.971631,0.963649,0.962456,0.975688
pr_1-5494467,0.987609,1.000000,0.995746,0.992228,0.992087,0.994719,0.993935,0.967687,0.969534,0.969128,0.987349,0.966123,0.966186,0.975457,0.975325,0.965959,0.966318,0.981262
pr_1-5497746,0.989740,0.995746,1.000000,0.994174,0.994048,0.996843,0.995876,0.969771,0.972106,0.969710,0.989077,0.968280,0.968287,0.977507,0.977386,0.968093,0.966898,0.982896
pr_1-5500177,0.986183,0.992228,0.994174,1.000000,0.994408,0.993132,0.995166,0.969519,0.970207,0.968298,0.985604,0.967699,0.966350,0.975962,0.975518,0.969086,0.966578,0.980598
pr_1-5500195,0.986011,0.992087,0.994048,0.994408,1.000000,0.995857,0.992232,0.967869,0.969941,0.968031,0.988341,0.966041,0.967433,0.977144,0.976736,0.967358,0.966172,0.982783


In [19]:
#========================================================================
# Get Low Corr Model For Ensemble
select_list = []
thres_corr = 0.985
base_model = top_corr.iloc[0].index[0]
select_list.append(base_model)

def ensemble_corr_checker(model):
    df_corr = top_corr.loc[model].reset_index()
    
    current_idx = list(top_corr.index).index(model)
    df_corr = df_corr.iloc[current_idx:, ]
    best_cv = 100
    best_corr = 0
    for _, rows in df_corr.iterrows():
        cv = rows[0]
        corr = rows[1]
        
        if cv not in select_list and corr<thres_corr:
            tmp = top_corr.loc[select_list, cv]
            cnt = tmp[tmp>=thres_corr].sum()
            if cnt==0:
                return cv, corr
    
    return '', 0

# cv, corr = ensemble_corr_checker(base_model)
# if corr>=thres_corr:
#     continue
    
is_no_stack = 0
is_loop = True
while is_loop:
    for model in select_list:    
        cv, corr = ensemble_corr_checker(base_model)
        if len(select_list)==1:
            if corr:
                select_list.append(cv)
        elif len(select_list)>1:
            cv, corr = ensemble_corr_checker(select_list[-1])
            if corr:
                select_list.append(cv)
            else:
                is_loop=False
#========================================================================

In [20]:
print(select_list)
df_select = top_corr.loc[select_list, select_list]
# tmp = df_select['lb3664'].iloc[:9]
# select_list = list(tmp.index)
# select_list
df_select

['pr_1-5486783', 'pr_1-5520199', 'pr_1-5520923', 'pr_1-5530868', 'pr_1-5540396']


,pr_1-5486783,pr_1-5520199,pr_1-5520923,pr_1-5530868,pr_1-5540396
pr_1-5486783,1.000000,0.965187,0.981394,0.971796,0.963649
pr_1-5520199,0.965187,1.000000,0.961573,0.970144,0.977088
pr_1-5520923,0.981394,0.961573,1.000000,0.969813,0.958947
pr_1-5530868,0.971796,0.970144,0.969813,1.000000,0.965666
pr_1-5540396,0.963649,0.977088,0.958947,0.965666,1.000000


In [22]:
# ================================================================
# 相関行列から移動するpathを決める
import shutil
for path in ens_list:
    for sel in select_list:
#         sel_key = sel[3:].replace('-', '.')
        sel_key = sel[3:]
        if path.count(sel_key):
#             shutil.move(path, '../ensemble/lgb_ensemble/')
            try:
#                 shutil.move(path, '../ensemble/dir_stack_blend/')
                shutil.move(path, '../ensemble/rm_outlier_ensemble/tmp/')
            except FileNotFoundError:
                pass
            pass
        else:
            pass
#             shutil.move(path, '../ensemble/used_stack/')
# ================================================================

100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,target,clf_pred,no_out_flg
card_id,,,
C_ID_92a2005557,-0.820283,0.000444,1.0
C_ID_3d0044924f,0.392913,0.007831,0.0
C_ID_d639edf6cd,0.688056,0.004074,0.0
C_ID_186d6a6901,0.142495,0.000797,0.0
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0
